In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os
os.environ['OMP_NUM_THREADS'] = '1'

import matplotlib.pyplot as plt

from kneed import KneeLocator 
from sklearn.cluster import DBSCAN

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta =   "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [3]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_clean_afi_sel_g_all_2.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_l = pickle.load( archivo ) 

In [4]:
inicio = time.time()
data_l = data_l.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[ True, True, True] )
data_l.reset_index(inplace=True)
data_l.rename(columns={'index': 'nuevo_indice'}, inplace=True)
data_l.drop(columns=['nuevo_indice'], inplace=True)
data_l['INDICE'] = data_l.index

In [5]:
#Casos de análisis
data = data_l[ (data_l['GRUPO_SEL']==1) ].copy()

In [6]:
#Algortimos K-means
cedula_1 = data.groupby('CEDULA_COD')['NUM_SEC_MES'].apply( lambda x: (x != 1).any() )
cedula_dist = cedula_1[ cedula_1 ].index
ul = data[ ~data['CEDULA_COD'].isin( cedula_dist )]
ml = data[  data['CEDULA_COD'].isin( cedula_dist )]

In [7]:
ul_dic = ul.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
ml_dic = ml.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')

In [ ]:
ul_dic[230]

In [13]:
from sklearn.neighbors import NearestNeighbors
def valor_epsilon( data, val ):
    
    for ced in data:
        aux = None
        aux = np.array( data[ced][val] ).reshape(-1, 1)
        eps = []
        minpt = []
        data[ced]['eps'] = []
        data[ced]['min_pt'] = []

        for i in range(1, 12):
            vecinos = NearestNeighbors( n_neighbors = min( i, len(aux) ) + 1  )
            vecinos_ajustado = vecinos.fit( aux )
            distancias = vecinos_ajustado.kneighbors(aux )[0]
            distancias1 = distancias[:, 1:]
            distancias_ordenadas = np.sort(distancias1.ravel())

            if len( np.unique( distancias_ordenadas ) )==1:
                eps.append( 0.0 )
                minpt.append( i + 1 )

            if len( np.unique( distancias_ordenadas ) )!=1:
                codo = KneeLocator(np.arange(len(distancias_ordenadas)), distancias_ordenadas, curve="convex", direction="increasing")

                eps.append( distancias_ordenadas[codo.elbow])
                minpt.append( i + 1 )

        for i, eps in enumerate( eps ):
            if eps not in data[ced]['eps']:
                data[ced]['eps'].append( eps )
                data[ced]['min_pt'].append( minpt[i] )
    

In [14]:
inicio = time.time()
valor_epsilon(ul_dic, 'SALARIO')
valor_epsilon(ml_dic, 'SALARIO')
fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

	Tiempo de ejecución es:  0.0  horas con  50.0  minutos y 33.89376521110535  segundos


In [15]:
nombre_archivo = 'viu_db_ul.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto1 = ul_dic

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto1, archivo) 
    
nombre_archivo = 'viu_db_ml.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto2 = ml_dic

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto2, archivo)  

In [3]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_db_ul.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    ul_dic = pickle.load( archivo ) 
    
directorio = r_ruta
nombre_archivo = 'viu_db_ml.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    ml_dic = pickle.load( archivo ) 

In [4]:
ul_dic

{126: {'SALARIO': [161.67,
   161.67,
   161.67,
   161.67,
   161.67,
   161.67,
   161.67,
   161.67,
   161.67,
   161.67,
   161.67,
   161.67,
   170.0,
   170.0,
   170.0,
   170.0,
   170.0,
   170.0,
   170.0,
   170.0,
   170.0,
   170.0,
   170.0,
   170.0,
   200.0,
   200.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   250.0,
   8.06],
  'INDICE': [13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64

In [64]:
def esti_eps_out( ul_dic, val ):
    for ced in ul_dic:
        aux =  np.array( ul_dic[ced][val] ).reshape(-1, 1)
        ep = ul_dic[ced]['eps']
        mt = ul_dic[ced]['min_pt']

        n_clusters = []
        n_outliers = []
        v_e = []

        for j in range(0, len(ep) ):

            if ep[ j ] == 0:
                n_clusters.append( -1 )
                n_outliers.append( -1 )
                v_e.append( ep[ j ] )

            if  ep[ j ]> 0 and  ep[ j ] <= 1:
                for eps in np.arange(0.0, ep[ j ], 0.20):
                    labels=None
                    db = DBSCAN( eps, min_samples = mt[j] ).fit( aux )
                    core_samples_mask = np.zeros_like( db.labels_, dtype=bool)
                    core_samples_mask[db.core_sample_indices_] = True
                    labels = db.labels_
                    n_clusters.append( len(set(labels)) - (1 if -1 in labels else 0) )
                    n_outliers.append( list(labels).count(-1) )
                    v_e.append( eps ) 

            if ep[ j ] > 1:
                ini =  max(0, ep[ j ] - 4 )
                for eps in np.arange( ini, ep[ j ], 1 ):
                    labels=None
                    db = DBSCAN(eps, min_samples = mt[ j ]).fit( aux )
                    core_samples_mask = np.zeros_like( db.labels_, dtype=bool)
                    core_samples_mask[db.core_sample_indices_] = True
                    labels = db.labels_
                    n_clusters.append( len(set(labels)) - (1 if -1 in labels else 0) )
                    n_outliers.append( list(labels).count(-1) )
                    v_e.append( eps ) 
 
        ag = np.empty((len(v_e) , 3))            
        ag[ :, 0 ] = v_e
        ag[ :, 1 ] = n_clusters
        ag[ :, 2 ] = n_outliers
        ul_dic[ced]['esti_eps_out'] = ag




In [65]:
val='SALARIO'
esti_eps_out( ul_dic, val )

ValueError: eps must be positive.

In [49]:
for j in range(0, len(ep) ):
    print(ep[j] )

0.0
153.60999999999999


In [ ]:
for eps in np.arange(0.50, 1.8, 0.20):
    db = DBSCAN(eps, min_samples=minPts).fit(features_norm)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_outliers = list(labels).count(-1)
    print ("%6.2f, %d, %d" % (eps, n_clusters_, n_outliers))

In [ ]:
ced=230
aux = None
data=ul_dic
val='SALARIO'
aux = np.array( data[ced][val] ).reshape(-1, 1)
eps = []
minpt = []
data[ced]['eps'] = []
data[ced]['min_pt'] = []

for i in range(1, len(aux)):

    vecinos = NearestNeighbors( n_neighbors = min( i, len(aux) ) + 1  )
    vecinos_ajustado = vecinos.fit( aux )
    distancias = vecinos_ajustado.kneighbors(aux )[0]
    distancias1 = distancias[:, 1:]
    distancias_ordenadas = np.sort(distancias1.ravel())

    if len( np.unique( aux ) )==1:
        eps.append( 0.0 )
        minpt.append( i + 1 )
    if len( np.unique( aux ) )==1:
        codo = KneeLocator( np.arange(len(distancias_ordenadas)), distancias_ordenadas, curve="convex", direction="increasing")

#     eps.append( distancias_ordenadas[codo.elbow])
#     minpt.append( i + 1 )

# for i, eps in enumerate( eps ):
#     if eps not in data[ced]['eps']:
#         data[ced]['eps'].append( eps )
#         data[ced]['min_pt'].append( minpt[i] )

In [ ]:
distancias_ordenadas

In [ ]:
codo = KneeLocator( np.arange(len(distancias_ordenadas)), distancias_ordenadas, curve="convex", direction="increasing")

In [ ]:
distancias_ordenadas[codo.elbow]

In [ ]:
ul_dic[230]

In [ ]:
aux = None
ced=230
val='SALARIO'
aux = np.array( data[ced][val] ).reshape(-1, 1)
eps = []
minpt = []

# data[ced]['eps'] = []
# data[ced]['min_pt'] = []

for i in range(1, len(aux)):

    vecinos = NearestNeighbors( n_neighbors = min( i, len(aux) ) + 1  )
    vecinos_ajustado = vecinos.fit( aux )
    distancias = vecinos_ajustado.kneighbors(aux )[0]
    distancias1 = distancias[:, 1:]
    distancias_ordenadas = np.sort(distancias1.ravel())

    if len(np.unique(distancias_ordenadas))==1:
        eps.append( 0.0 )
        minpt.append( i + 1 )

    if len(np.unique(distancias_ordenadas))!=1:
        codo = KneeLocator(np.arange(len(distancias_ordenadas)), distancias_ordenadas, curve="convex", direction="increasing")

        eps.append( distancias_ordenadas[codo.elbow])
        minpt.append( i + 1 )

# for i, eps in enumerate( eps ):
#     if eps not in data[ced]['eps']:
#         data[ced]['eps'].append( eps )
#         data[ced]['min_pt'].append( minpt[i] )

In [ ]:
distancias_ordenadas

In [ ]:
param_grid = {
    'eps': list( [ valo_epsilon( 126, ul_dic, x, 'SALARIO') for x in range(1, 60)] ),
    'min_samples': list(range(3, 60))
}
resultado = {}
for i, eps in enumerate(param_grid['eps']):
    if eps not in resultado:
        resultado[eps] = param_grid['min_samples'][i]
resultado

In [ ]:
for eps in np.arange(145, 240, 5):
    db = DBSCAN(eps, min_samples=17).fit(aux)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_outliers = list(labels).count(-1)
    print ("%6.2f, %d, %d" % (eps, n_clusters_, n_outliers))

In [ ]:
db = DBSCAN(eps=145, min_samples=3).fit(aux)
labels = db.labels_
ul_dic[126]['ATI_DB'] = labels.tolist()
labels 

In [ ]:
salarios = ul_dic[126]['SALARIO']
indices = ul_dic[126]['INDICE']
ati_db = ul_dic[126]['ATI_DB']

# Crear un scatter plot
plt.scatter(indices, salarios, c=ati_db, cmap='coolwarm', edgecolor='k')

# Añadir etiquetas y título
plt.xlabel('INDICE')
plt.ylabel('SALARIO')
plt.title('Salario vs. Índice coloreado por ATI_DB')

# Mostrar la barra de colores
plt.colorbar(label='ATI_DB')

# Mostrar la gráfica
plt.show()

In [ ]:
ul_dic

In [ ]:
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator 

def epsilon_dbscan(data, minPts, val):
    data = data[val].to_numpy()
    data = data.reshape(-1, 1)
    
    # Modelo de vecinos más cercanos
    vecinos = NearestNeighbors(n_neighbors= minPts)
    vecinos_ajustado = vecinos.fit(data)
    distancias = vecinos_ajustado.kneighbors(data)[0]

    # Ordenar distancias
    distancias_ordenadas = np.sort(distancias[: ,(minPts-1)])

    # Detección del codo
    num_codo = KneeLocator(range(0, data.shape[0]), distancias_ordenadas, curve="convex", direction="increasing").elbow
    
    return(distancias_ordenadas[num_codo])

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import StandardScaler

data_aux = data_l[(data_l['CEDULA_COD'] == 20612) & (data_l['GRUPO_SEL']==1)]
data_AUX = data_aux.copy()
data = data_AUX['SALARIO'].to_numpy()
me = data.mean()
st = data.std(0)
data_AUX['SALARIO'] = ( data - me ) / st
data =np.array(data_AUX['SALARIO']).reshape(-1, 1)

param_grid = {
    'eps': list(set([epsilon_dbscan(data_AUX, x, 'SALARIO') for x in range(3, 18)]) - {0}),
    'min_samples': [  2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
}
param_grid

epsilon_dbscan(data_AUX, 2, 'SALARIO')